Towards Zero-Shot Knowledge Distillation for Natural
Language Processing对比方法
5个epoch
20000个ood数据
没有学习率递减
没有加上bert的 cls和sep符号

In [1]:
import numpy as np
import torch
import torch.nn as nn
from data_init import class_incremental
from model_config import MyModel_Config
from pytorch_pretrained_bert import BertConfig, BertTokenizer, BertModel, BertForMaskedLM
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt   #jupyter要matplotlib.pyplot
from torch.utils.data import DataLoader, Dataset, TensorDataset
import random
from train_eval import Model_Train
import re
from tqdm import *
import pandas as pd
from pandas import DataFrame
from torch.distributions import Dirichlet

/root/deepo/LYL/anaconda/ls/envs/csuse/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
'''调整随机数'''
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(5)

device = 'cuda:1'

In [3]:
'''利用增量数据初始化处理数据集'''
train_incremental = class_incremental('./data/snips/train.csv', 'csv', 64, 7, 20, True, 2000) #最后的500表示每个类的数据个数限制
train_original_datas, train_incremental_datas_list, train_original_labels, train_all_incremental_labels, labels, label_to_idx = train_incremental.prepare_for_incremental()
train_Joint_datasets = train_incremental.Joint_incremental()

#初始化验证集的原始类和增量类数据
dev_incremental = class_incremental('./data/snips/valid.csv', 'csv', 64, 7, 20, True, 1000, 'eval', labels, label_to_idx)
dev_original_datas, dev_incremental_datas_list, dev_original_labels, dev_all_incremental_labels = dev_incremental.prepare_for_incremental()
dev_Joint_datasets = dev_incremental.Joint_incremental()

#初始化测试集的原始类和增量类数据
test_incremental = class_incremental('./data/snips/test.csv', 'csv', 64, 7, 20, True, 1000, 'eval', labels, label_to_idx)
test_original_datas, test_incremental_datas_list, test_original_labels, test_all_incremental_labels = test_incremental.prepare_for_incremental()
test_Joint_datasets = test_incremental.Joint_incremental()

In [4]:
'''定义训练模型'''
class Teachermodel(nn.Module):
    def __init__(self,config):
        super(Teachermodel,self).__init__()
        self.bert=BertModel.from_pretrained(config.bert_path)  
        for param in self.bert.parameters():
            param.requires_grad = True
 
        
        self.dropout=nn.Dropout(config.dropout)

        self.fc = nn.Linear(768, config.num_classes ) 


    def forward(self, tokens):

        encoder_out,pooled = self.bert(tokens,output_all_encoded_layers=False) 
     
        out=self.fc(pooled)
 
        out = self.dropout(out)
        
        return out



'''定义Bert生成模型。通过在一个OOD数据集训练让其输入噪声后可以输出在该OOD上的词汇集'''
class Bert_Gen(nn.Module):  #Bert生成模型
    def __init__(self, Bert_config): #tokens_len表示生成数据的长度,即最后bert要生成这么长的文本
        super(Bert_Gen,self).__init__()
        self.device = Bert_config.device
        
        self.fc1 = nn.Linear(1,30522)
        
        self.bert1=BertModel.from_pretrained(Bert_config.bertmini_path)  #从路径加载预训练模型
        for param in self.bert1.parameters():
            param.requires_grad = True # 使参数可更新
        
        self.fc3= nn.Linear(Bert_config.hidden_size, 1024)
        self.fc4 = nn.Linear(1024, 30522)
    
    def forward(self, z): #tokens表示dataloader中的一个batch的tokens，即去掉label部分的token tensor
        out = self.fc1(z*0.01)
        out = F.gumbel_softmax(out, 10, True)
        out = torch.argmax(out, dim=1)

        out = out.long()#将经过线性层将正态分布z变为long型整数输入到bert
        
        encoder_out, pool = self.bert1(out.view(1,-1), output_all_encoded_layers=False) #得到每个token的向量表示
        
        out = self.fc3(encoder_out.squeeze())  
        out = F.relu(out)
        out = self.fc4(out)  #经过线性层处理生成新的向量，和bert词表大小相同
        out = F.gumbel_softmax(out, 10, True)
        
        return out  #输出一个batch的softmax [batch_size, 类别的softmax得分]
        

'''student模型'''
class Bert_student(nn.Module):
    def __init__(self,config):
        super(Bert_student,self).__init__()
        self.bert=BertModel.from_pretrained(config.bertmini_path)  
        for param in self.bert.parameters():
            param.requires_grad = True
        
        self.dropout=nn.Dropout(config.dropout)
        self.fc = nn.Linear(256, config.num_classes ) 


    def forward(self, tokens):

    
        encoder_out,pooled = self.bert(tokens,output_all_encoded_layers=False) 
        out=self.fc(pooled)
        out = self.dropout(out)
        
        return out

In [5]:
'''读取OOD的wiki103数据'''
import os
def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, encoding='utf-8') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

paragraphs = _read_wiki('./wiki103')

In [6]:
'''将读取的wiki数据tokenize'''
tokenizer = BertTokenizer.from_pretrained('./bert-pretrained')
tokens = []
datas_size = len(paragraphs)
for l in tqdm(range(datas_size)):
    for k in range(len(paragraphs[l])):
        text = paragraphs[l][k]
        text = tokenizer.tokenize(text)
        token = tokenizer.convert_tokens_to_ids(text)
        tokens.append(token)

100%|██████████| 760542/760542 [20:57<00:00, 604.83it/s]


In [7]:
'''利用原始数据训练并保存Teacher model'''
print('*******************原始数据训练Teacher model**********************')
teacher_model = Teachermodel(MyModel_Config(train_original_labels))
optimizer = torch.optim.AdamW(teacher_model.parameters(), lr=1e-5)
loss_fuc = nn.CrossEntropyLoss()

'''训练模型，得到精度'''
accuracy_train, loss_train = Model_Train().my_train(teacher_model, loss_fuc, optimizer, 10, train_original_datas)
accuracy_dev, loss_dev = Model_Train().eval_for_incremental(teacher_model, dev_original_datas, loss_fuc)
accuracy_test, loss_test = Model_Train().eval_for_incremental(teacher_model, test_original_datas, loss_fuc)
print('训练集精度变化'+str(accuracy_train))
print('验证集最终精度'+str(accuracy_dev))
print('测试集最终精度'+str(accuracy_test))

*******************原始数据训练Teacher model**********************


100%|██████████| 10/10 [24:52<00:00, 149.28s/it]


训练集精度变化[0.9859068627450981, 0.9905790441176471, 0.9931832107843137, 0.9954810049019608, 0.9931832107843137, 0.9967830882352942, 0.9983915441176471, 0.9983149509803921, 0.9990042892156863, 0.9993872549019608]
验证集最终精度0.9875
测试集最终精度0.98125


In [13]:
'''对抗性训练student模型
参数：
Bert_Gen: the model which was trained by the ood data(wiki103), and can gennerate tokens which are similar to the ood data
Bert_CNN: the teacher model pretrained on the original training set
Bert_CNN_Student: the student model which we will train
ood_datas: the list consists of wiki103 data which were tokenized, the length of the elements data can be diffierent
optimizer1: the optimizier that optimizes the Bert_Gen model
optimizer2: the optimizer that optimizes the Bert_CNN_Student
loss_func1: Cross Entropy loss
loss_func2:KLdiv loss
batch_size: when batch_size losses are sumed, then loss_backward() 
nG: the adversial epochs
nS: the epochs that the student model will be trained
'''
def adverse_train(Bert_Gen, Teacher_model, Student_model, ood_datas, optimizer1, optimizer2, loss_func1, loss_func2, train_original_datas, dev_original_datas, test_original_datas, batch_size = 64, nG = 10, nS=20, temper=10, device = 'cuda:1'):

    Bert_Gen = Bert_Gen.to(device)
    Teacher_model = Teacher_model.to(device)
    Student_model = Student_model.to(device)
    '''对抗性训练，训练BERtGEN生成使得teachermodel和studentmodel之间KL散度大的样本'''
    Student_model.eval()
    Teacher_model.eval()
    Bert_Gen.train()
    scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer1, gamma=0.99)#每一轮epoch学习率递减
    scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer2, gamma=0.99)#每一轮epoch学习率递减
    for i in tqdm(range(nG)):
        loss = 0
        for j in range(len(ood_datas)):
            ood_data = torch.tensor(ood_datas[j]).to(device) #list的数据要转化为tensor形式
            z = torch.randn(len(ood_data), 1).to(device)  #噪声数据，和data的长度相同
            optimizer1.zero_grad()
            
            probs = Bert_Gen(z)
            
            tokens_gen = torch.argmax(probs, dim=1).view(1,-1).long() #输入Bert的格式
            loss = loss + 1/2*loss_func1(probs, ood_data) #保证z生成的tokens接近wiki的ood数据集
            
            tokens = tokens_gen.squeeze().tolist()
            tokens.append(102)#添加sep符号
            tokens.insert(0, 101) #添加cls符号
            tokens_gen = torch.tensor(tokens).view(1,-1).to(device).long()  #感觉加了之后要好一些
            
            with torch.no_grad():
                prob_student = Student_model(tokens_gen)
                prob_teacher = Teacher_model(tokens_gen)
            loss = loss - 1/2*loss_func2(F.log_softmax(prob_student / temper, dim=1), F.softmax(prob_teacher / temper, dim=1)) 
            #训练BertGen生成使得student和teacher散度大的ood样本
            
            if j>0 and j%batch_size == 0:  #起到batchsize作用，每64个优化一次
                
                loss /= batch_size
                loss_num = loss.item()
                loss.backward()
                optimizer1.step()
                #scheduler1.step()#学习率递减
                loss = 0  #原本的batch情况是loss直接赋值，所以这里要将loss重置为0
                #这里我亲自尝试过，如果要使得64次优化一次和batch效果相同的话，要将loss除以64
            
    '''训练studentmodel使之在之前训练的BertGen生成的样本和ood样本上的KL散度尽可能小'''     
    Student_model.train()
    Teacher_model.eval()
    Bert_Gen.eval()       
    max_acc_fin = 0

    for i in tqdm(range(nS)):
        loss = 0
        CE_loss = nn.CrossEntropyLoss()
        max_acc_test = 0

        for j in range(len(ood_datas)):
            ood_data = torch.tensor(ood_datas[j]).to(device) #list的数据要转化为tensor形式
            z = torch.randn(len(ood_data), 1).to(device)  #噪声数据，和data的长度相同
            ood_data = ood_data.view(1,-1)
            optimizer2.zero_grad()
            
            with torch.no_grad():
                probs = Bert_Gen(z)
                tokens_gen = torch.argmax(probs, dim=1).view(1,-1).long() #输入Bert的格式
                tokens = tokens_gen.squeeze().tolist()
                tokens.append(102)#添加sep符号
                tokens.insert(0, 101) #添加cls符号
                tokens_gen = torch.tensor(tokens).view(1,-1).to(device).long()  #感觉加了之后要好一些
                prob_teacher = Teacher_model(ood_data)
                prob_teacher_gen = Teacher_model(tokens_gen)
            
            
            prob_student = Student_model(ood_data)
            prob_student_gen = Student_model(tokens_gen)
            loss = loss + 0.2*loss_func2(F.log_softmax(prob_student_gen / temper, dim=1), F.softmax(prob_teacher_gen / temper, dim=1)) + (1-0.2)*loss_func2(F.log_softmax(prob_student / temper, dim=1), F.softmax(prob_teacher / temper, dim=1))
            
            
            if j>0 and j%batch_size == 0:
                loss /= batch_size
                loss_num = loss.item()
                loss.backward()
                optimizer2.step()
                #scheduler2.step()#学习率递减
                loss = 0

                Student_model.eval()
                accuracy_test, _ = Model_Train().eval_for_incremental(Student_model, test_original_datas, CE_loss)

                if max_acc_test<accuracy_test:  
                                             #这之后是求每一个Ns批次的最大测试集精度
                    max_acc_test = accuracy_test
                    accuracy_train, _ = Model_Train().eval_for_incremental(Student_model, train_original_datas, CE_loss)
                    accuracy_dev, _ = Model_Train().eval_for_incremental(Student_model, dev_original_datas, CE_loss)
            
            if j>0 and j%batch_size == 0:
                print('训练集精度'+str(accuracy_train))
                print('验证集精度'+str(accuracy_dev))
                print('测试集精度'+str(max_acc_test))

            if j>0 and j%batch_size == 0 and max_acc_fin<max_acc_test:
                max_acc_fin = max_acc_test
                accuracy_train_fin = accuracy_train
                accuracy_dev_fin = accuracy_dev

            Student_model.train()
    
        print('训练集最终精度'+str(accuracy_train_fin))
        print('验证集最终精度'+str(accuracy_dev_fin))
        print('测试集最终精度'+str(max_acc_fin))


        

In [14]:
bert_gen = torch.load('./model/bert_genMINI3')
teacher_model = teacher_model
student_model = Bert_student(MyModel_Config(train_original_labels))

optimizer1 = torch.optim.AdamW(bert_gen.parameters(), lr=1e-5)
optimizer2 = torch.optim.AdamW(student_model.parameters(), lr=1e-4)

loss_func1 = nn.CrossEntropyLoss()
loss_func2 = nn.KLDivLoss(reduction='mean')

ood_num = 20000
ood_datas = []
for i in range(ood_num):  #Bert最长读512最少读2长度的tokens,所以要处理
    if len(tokens[i]) <100 and len(tokens[i])>25:
        ood_datas.append(tokens[i])
print(len(ood_datas))
    
N_k=3
for k in range(N_k):
    adverse_train(bert_gen, teacher_model, student_model, ood_datas, optimizer1, optimizer2, loss_func1, loss_func2, train_original_datas, dev_original_datas, test_original_datas)
    accuracy_train, _ = Model_Train().eval_for_incremental(student_model, train_original_datas, loss_func1)
    accuracy_dev, _ = Model_Train().eval_for_incremental(student_model, dev_original_datas, loss_func1)
    accuracy_test, _ = Model_Train().eval_for_incremental(student_model, test_original_datas, loss_func1)
    print('训练集精度变化'+str(accuracy_train))
    print('验证集最终精度'+str(accuracy_dev))
    print('测试集最终精度'+str(accuracy_test))


10717


  0%|          | 0/20 [00:00<?, ?it/s]

训练集精度0.15019914215686275
验证集精度0.165625
测试集精度0.1625
训练集精度0.15019914215686275
验证集精度0.165625
测试集精度0.1625
训练集精度0.16957720588235295
验证集精度0.196875
测试集精度0.1796875
训练集精度0.19132965686274508
验证集精度0.2140625
测试集精度0.1875
训练集精度0.19722732843137256
验证集精度0.2078125
测试集精度0.190625
训练集精度0.1998314950980392
验证集精度0.209375
测试集精度0.2
训练集精度0.1998314950980392
验证集精度0.209375
测试集精度0.2
训练集精度0.20557598039215685
验证集精度0.2296875
测试集精度0.2015625
训练集精度0.20955882352941177
验证集精度0.2484375
测试集精度0.203125
训练集精度0.20932904411764705
验证集精度0.25
测试集精度0.2140625
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.20595894607843138
验证集精度0.246875
测试集精度0.228125
训练集精度0.2

  5%|▌         | 1/20 [16:25<5:11:58, 985.16s/it]

训练集最终精度0.40349264705882354
验证集最终精度0.4
测试集最终精度0.3875
训练集精度0.3516390931372549
验证集精度0.3640625
测试集精度0.35
训练集精度0.3516390931372549
验证集精度0.3640625
测试集精度0.35
训练集精度0.3516390931372549
验证集精度0.3640625
测试集精度0.35
训练集精度0.3516390931372549
验证集精度0.3640625
测试集精度0.35
训练集精度0.3516390931372549
验证集精度0.3640625
测试集精度0.35
训练集精度0.3516390931372549
验证集精度0.3640625
测试集精度0.35
训练集精度0.3516390931372549
验证集精度0.3640625
测试集精度0.35
训练集精度0.3516390931372549
验证集精度0.3640625
测试集精度0.35
训练集精度0.3902420343137255
验证集精度0.3765625
测试集精度0.36875
训练集精度0.42394301470588236
验证集精度0.3921875
测试集精度0.4015625
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425
训练集精度0.4417126225490196
验证集精度0.4234375
测试集精度0.425

...

训练集最终精度0.6158854166666666
验证集最终精度0.7296875
测试集最终精度0.69375
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865
验证集精度0.5296875
测试集精度0.465625
训练集精度0.40732230392156865

 85%|████████▌ | 17/20 [3:54:14<41:05, 821.97s/it]

训练集最终精度0.6158854166666666
验证集最终精度0.7296875
测试集最终精度0.69375
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精度0.4625
训练集精度0.4053308823529412
验证集精度0.521875
测试集精

 85%|████████▌ | 17/20 [4:02:22<42:46, 855.46s/it]


KeyboardInterrupt: 